In [1]:
import pandas as pd
import time
import numpy as np
from sklearn.cross_validation import train_test_split
# import lightgbm as lgb
from pandas_summary import DataFrameSummary
pd.options.display.max_columns = 100
pd.options.display.max_rows = 50
from pathlib import Path
from fastai.structured import *

/home/channoklam_cnl/anaconda3/envs/fastai/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
import torch

## Pre-Processing

In [8]:
dtypes = {
            'ip'            : 'uint32',
            'app'           : 'uint16',
            'device'        : 'uint16',
            'os'            : 'uint16',
            'channel'       : 'uint16',
            'is_attributed' : 'uint8',
            'click_id'      : 'uint32'
            }

In [4]:
def prepare_data(sample = False):
    if sample == False: nrows = None
    else: nrows = 100000
    
    
    path = Path('.')

    dtypes = {
            'ip'            : 'uint32',
            'app'           : 'uint16',
            'device'        : 'uint16',
            'os'            : 'uint16',
            'channel'       : 'uint16',
            'is_attributed' : 'uint8',
            'click_id'      : 'uint32'
            }

    print('load train...')
    train_df = pd.read_csv("train.csv.zip", nrows=nrows, dtype=dtypes, usecols=['ip','app','device','os', 'channel', 'click_time', 'is_attributed'])
    print('load test...')
    test_df = pd.read_csv("test.csv.zip", nrows=nrows, dtype=dtypes, usecols=['ip','app','device','os', 'channel', 'click_time', 'click_id'])
    
    return train_df, test_df

In [5]:
sample = False

In [6]:
train_df, test_df = prepare_data(sample = sample)

load train...


ParserError: Error tokenizing data. C error: Calling read(nbytes) on source failed. Try engine='python'.

In [ ]:
train_df.shape, test_df.shape

In [ ]:
train_df.head()

In [ ]:
train_df.click_time.max()

In [ ]:
train_df.click_time.min()

In [ ]:
train_df.memory_usage()/1028/1028

In [ ]:
test_df.memory_usage()/1028/1028

In [ ]:
train_df.shape

In [ ]:
train_df.dtypes

In [ ]:
train_df.click_time = pd.to_datetime(train_df.click_time)
test_df.click_time = pd.to_datetime(test_df.click_time)

In [9]:
def add_datepart(df, fldname, drop=True):

    fld = df[fldname]
    if not np.issubdtype(fld.dtype, np.datetime64):
        df[fldname] = fld = pd.to_datetime(fld, infer_datetime_format=True)
    targ_pre = re.sub('[Dd]ate$', '', fldname)
    for n in ( 'Day','Hour','Minute','Second'
             ):
        df[targ_pre+n] = getattr(fld.dt,n.lower()).astype(np.int8)
    df[targ_pre+'Elapsed'] = fld.astype(np.int64) // 10**9
    if drop: df.drop(fldname, axis=1, inplace=True)

In [ ]:
# tables = [train_df, test_df]
# from IPython.display import HTML
# for t in tables:
#     df = DataFrameSummary(t)
#     display(t.head())
#     display(df.summary())

In [ ]:
# train_df.click_time.describe()

In [ ]:
# test_df.click_time.describe()

In [ ]:
test_df.memory_usage().sum()/1028/1028

In [ ]:
train_df.memory_usage().sum()/1028/1028

In [ ]:
import gc
import re

n_train = len(train_df)
train_df=train_df.append(test_df)

del test_df
gc.collect()

In [ ]:
train_df.memory_usage()/1024/1024

In [ ]:
train_df['is_attributed'].fillna(-1,inplace=True)
train_df['is_attributed'] = train_df['is_attributed'].astype(np.int8);

In [ ]:
train_df['click_id'].fillna(-1,inplace=True)
train_df['click_id'] = train_df['click_id'].astype(np.int8)

In [ ]:
train_df.memory_usage()/1024/1024

In [ ]:
# train_df.to_csv('tmp.csv',index=False)

In [ ]:
train_df.memory_usage().sum()/1028/1028

In [13]:
train_df[0].memory_usage()/1024/1024

Index                 0.000076
app                  19.073486
channel              19.073486
click_id             38.146973
device               19.073486
ip                   38.146973
is_attributed         9.536743
os                   19.073486
click_timeDay         9.536743
click_timeHour        9.536743
click_timeMinute      9.536743
click_timeSecond      9.536743
click_timeElapsed    76.293945
dtype: float64

In [20]:
import gc

In [15]:
train_df_chunk = pd.read_csv('tmp.csv', chunksize = 10000000, dtype=dtypes)

train_df = []
for i ,chunk in enumerate(train_df_chunk):
    print(i)
    %time add_datepart(chunk,'click_time')
    display(chunk.head())
    train_df.append(chunk)
    gc.collect()
print('done')




0
CPU times: user 14.1 s, sys: 235 ms, total: 14.3 s
Wall time: 3.96 s


,app,channel,click_id,device,ip,is_attributed,os,click_timeDay,click_timeHour,click_timeMinute,click_timeSecond,click_timeElapsed
0,3,379,4294967295,1,83230,0,13,6,14,32,21,1509978741
1,3,379,4294967295,1,17357,0,19,6,14,33,34,1509978814
2,3,379,4294967295,1,35810,0,13,6,14,34,12,1509978852
3,14,478,4294967295,1,45745,0,13,6,14,34,52,1509978892
4,3,379,4294967295,1,161007,0,13,6,14,35,8,1509978908


1
CPU times: user 13.9 s, sys: 243 ms, total: 14.2 s
Wall time: 3.92 s


,app,channel,click_id,device,ip,is_attributed,os,click_timeDay,click_timeHour,click_timeMinute,click_timeSecond,click_timeElapsed
10000000,15,245,4294967295,1,117021,0,22,7,0,12,3,1510013523
10000001,3,211,4294967295,1,5348,0,13,7,0,12,3,1510013523
10000002,2,435,4294967295,1,27815,0,19,7,0,12,3,1510013523
10000003,1,118,4294967295,1,80306,0,22,7,0,12,3,1510013523
10000004,3,489,4294967295,1,115066,0,13,7,0,12,4,1510013524


2
CPU times: user 14.7 s, sys: 351 ms, total: 15 s
Wall time: 4.25 s


...

19
CPU times: user 14.9 s, sys: 1.96 s, total: 16.9 s
Wall time: 4.53 s


,app,channel,click_id,device,ip,is_attributed,os,click_timeDay,click_timeHour,click_timeMinute,click_timeSecond,click_timeElapsed
190000000,2,205,4294967214,1,52076,255,19,10,5,35,28,1510292128
190000001,2,122,4294967215,1,69810,255,15,10,5,35,28,1510292128
190000002,25,259,4294967216,1,10201,255,19,10,5,35,28,1510292128
190000003,3,424,4294967217,1,50924,255,13,10,5,35,28,1510292128
190000004,9,489,4294967218,1,30840,255,16,10,5,35,28,1510292128


20
CPU times: user 8.59 s, sys: 136 ms, total: 8.72 s
Wall time: 1.47 s


,app,channel,click_id,device,ip,is_attributed,os,click_timeDay,click_timeHour,click_timeMinute,click_timeSecond,click_timeElapsed
200000000,12,265,46,1,34980,255,19,10,13,52,17,1510321937
200000001,3,173,47,1,6367,255,13,10,13,52,17,1510321937
200000002,19,347,48,0,7838,255,50,10,13,52,17,1510321937
200000003,2,212,49,1,12788,255,16,10,13,52,17,1510321937
200000004,2,435,50,1,35414,255,19,10,13,52,17,1510321937


done


In [21]:
train_df = pd.concat(train_df)

In [25]:
train_df.memory_usage().sum()/1024/1024

5633.484354972839

In [ ]:
# del test_df
gc.collect()

In [ ]:
# DataFrameSummary(train_df).summary()

Looks like the data is pretty good, no missing data issue or it may have be removed already.

In [29]:
# # of clicks for each ip-day-hour combination
print('group by...')
gp = train_df[['ip','click_timeDay','click_timeHour','click_timeMinute','channel']].groupby(by=['ip','click_timeDay','click_timeHour','click_timeMinute'])[['channel']].count().reset_index().rename(index=str, columns={'channel': 'qty'})
print('merge...')
train_df = train_df.merge(gp, on=['ip','click_timeDay','click_timeHour','click_timeMinute'], how='left')
del gp
gc.collect()

group by...
merge...


1097

In [30]:
# # of clicks for each ip-app combination
print('group by...')
gp = train_df[['ip','app', 'channel']].groupby(by=['ip', 'app'])[['channel']].count().reset_index().rename(index=str, columns={'channel': 'ip_app_count'})
train_df = train_df.merge(gp, on=['ip','app'], how='left')
del gp
gc.collect()

group by...


98

In [31]:
# # of clicks for each ip-app-os combination
print('group by...')
gp = train_df[['ip','app', 'os', 'channel']].groupby(by=['ip', 'app', 'os'])[['channel']].count().reset_index().rename(index=str, columns={'channel': 'ip_app_os_count'})
train_df = train_df.merge(gp, on=['ip','app', 'os'], how='left')
del gp
gc.collect()

group by...


114

In [32]:
print("vars and data type: ")
train_df.info()
train_df['qty'] = train_df['qty'].astype('uint16')
train_df['ip_app_count'] = train_df['ip_app_count'].astype('uint16')
train_df['ip_app_os_count'] = train_df['ip_app_os_count'].astype('uint16')

vars and data type: 
<class 'pandas.core.frame.DataFrame'>
Int64Index: 203694359 entries, 0 to 203694358
Data columns (total 15 columns):
app                  uint16
channel              uint16
click_id             uint32
device               uint16
ip                   uint32
is_attributed        uint8
os                   uint16
click_timeDay        int8
click_timeHour       int8
click_timeMinute     int8
click_timeSecond     int8
click_timeElapsed    int64
qty                  int64
ip_app_count         int64
ip_app_os_count      int64
dtypes: int64(4), int8(4), uint16(4), uint32(2), uint8(1)
memory usage: 11.6 GB


In [33]:
train_df.head(5)

,app,channel,click_id,device,ip,is_attributed,os,click_timeDay,click_timeHour,click_timeMinute,click_timeSecond,click_timeElapsed,qty,ip_app_count,ip_app_os_count
0,3,379,4294967295,1,83230,0,13,6,14,32,21,1509978741,1,5759,1431
1,3,379,4294967295,1,17357,0,19,6,14,33,34,1509978814,1,5245,1451
2,3,379,4294967295,1,35810,0,13,6,14,34,12,1509978852,1,2156,462
3,14,478,4294967295,1,45745,0,13,6,14,34,52,1509978892,1,10547,2186
4,3,379,4294967295,1,161007,0,13,6,14,35,8,1509978908,1,232,80


In [36]:
for i in [  'Day','Hour','Minute','Second']:
    train_df['click_time'+i] = train_df['click_time'+i].astype(np.int16)

In [37]:
train_df.memory_usage().sum()/1024/1024

9130.129693031311

In [42]:
n_train = len(train_df)

In [43]:
test_df = train_df[n_train:]
val_df = train_df[int(n_train*0.75):n_train]
train_df = train_df[:int(n_train*0.75)]

In [ ]:
with open ('train_df.pkl','wb') as f:
    pickle.dump(train_df,f)

In [47]:
with open ('val_df.pkl','wb') as f:
    pickle.dump(val_df,f)

OSError: [Errno 28] No space left on device

In [46]:
with open ('test_df.pkl','wb') as f:
    pickle.dump(test_df,f)    

In [ ]:
def over_sample(df):
    count_class_0, count_class_1 = df.is_attributed.value_counts()

    # Divide by class
    df_class_0 = df[df['is_attributed'] == 0]
    df_class_1 = df[df['is_attributed'] == 1]
    df_class_1_over = df_class_1.sample(count_class_0, replace=True)
    df = pd.concat([df_class_0, df_class_1_over], axis=0)

    print('Random over-sampling:')
    display(df.is_attributed.value_counts().plot(kind='bar', title='Count (target)'))
    return df
    


In [ ]:
train_df = over_sample(train_df)

In [ ]:
# # Class count
# count_class_0, count_class_1 = val_df.is_attributed.value_counts()

# # Divide by class
# df_class_0 = val_df[val_df['is_attributed'] == 0]
# df_class_1 = val_df[val_df['is_attributed'] == 1]

# # df_class_0.head()

# df_class_1_over = df_class_1.sample(count_class_0, replace=True)
# val_df = pd.concat([df_class_0, df_class_1_over], axis=0)

# print('Random over-sampling:')
# val_df.is_attributed.value_counts().plot(kind='bar', title='Count (target)');

In [ ]:
print("train size: ", len(train_df))
print("valid size: ", len(val_df))
print("test size : ", len(test_df))

## Create Feature

In [ ]:
train_df.head(2)

In [ ]:
train_df.columns

In [ ]:
cat_vars = ['app', 'channel', 'device', 'ip', 'os',
       'click_timeYear', 'click_timeMonth', 'click_timeWeek', 'click_timeDay',
       'click_timeDayofweek', 'click_timeDayofyear', 'click_timeHour',
       'click_timeMinute', 'click_timeSecond', 'click_timeIs_month_end',
       'click_timeIs_month_start', 'click_timeIs_quarter_end',
       'click_timeIs_quarter_start', 'click_timeIs_year_end',
       'click_timeIs_year_start']

contin_vars = ['qty', 'ip_app_count',
       'ip_app_os_count']

n = len(train_df); n

In [ ]:
dep = 'is_attributed'
joined = train_df[cat_vars+contin_vars+[dep]].copy()

In [ ]:
test_df.columns

In [ ]:
test_df[dep] = 0
joined_test = test_df[cat_vars+contin_vars+[dep, 'click_id', 'click_timeElapsed']].copy()

In [ ]:
for v in cat_vars: joined[v] = joined[v].astype('category').cat.as_ordered()

In [ ]:
apply_cats(joined_test, joined)

In [ ]:
for v in contin_vars:
    joined[v] = joined[v].astype('float32')
    joined_test[v] = joined_test[v].astype('float32')

In [ ]:
df, y, nas, mapper = proc_df(joined, 'is_attributed', do_scale=True)
yl = np.log(y)

In [ ]:
y = y.astype(np.int64)

In [ ]:
torch.Tensor(y).type()

In [ ]:
df_test, _, nas, mapper = proc_df(joined_test, 'is_attributed', do_scale=True, skip_flds=['click_id'],
                                  mapper=mapper, na_dict=nas)

In [ ]:
df_test = df_test.set_index("click_timeElapsed")

In [ ]:
df.columns

In [ ]:
df_test.columns

## DL

In [ ]:
from fastai.column_data import *

In [ ]:
path

In [ ]:
np.arange(len(joined))[int(len(joined)*0.8) :]

In [ ]:
import pdb
# pdb.set_trace()
md = ColumnarModelData.from_data_frame(path,[-1], df, y.astype(np.int64), cat_flds=cat_vars, bs=64,
                                       test_df=None,is_reg=False, is_multi=False)

In [ ]:
cat_sz = [(c, len(joined[c].cat.categories)+1) for c in cat_vars]

In [ ]:
emb_szs = [(c, min(50, (c+1)//2)) for _,c in cat_sz]

In [ ]:
torch.cuda.is_available()

In [ ]:
a = torch.Tensor(10)

In [ ]:
torch.cuda.current_device()

In [ ]:
USE_GPU = False

In [ ]:
m = md.get_learner(emb_szs, len(df.columns)-len(cat_vars),
                   0.04, 1, [1000,500], [0.001,0.01], y_range=None)
lr = 1e-3

In [ ]:
tmp = iter(md.trn_dl)

In [ ]:
*xs,ys = next(tmp)

In [ ]:
ys.long

In [ ]:
len(xs)

In [ ]:
len(cat_vars)

In [ ]:
len(contin_vars)

In [ ]:
for i in xs:
    print(i.size())

In [ ]:
m.model.training

In [ ]:
m.model.state_dict

In [ ]:
m.lr_find()

In [ ]:
m.sched.plot(0,0)

In [ ]:
4 